# Математика ансамблевых методов

In [1]:
import numpy as np
x = np.array([4.4, 3.8, 4.8, 4.0, 3.4, 4.2, 5.2])
x.var().round(3)

0.317

In [2]:
import pandas as pd

df = pd.read_csv('wineQualityReds.csv')
df = df.drop(['Unnamed: 0'], axis=1)
df

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [3]:
df['quality'] = df['quality'].apply(lambda x: 1 if x >= 6 else 0)
df.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [4]:
df.isnull().mean()

fixed.acidity           0.0
volatile.acidity        0.0
citric.acid             0.0
residual.sugar          0.0
chlorides               0.0
free.sulfur.dioxide     0.0
total.sulfur.dioxide    0.0
density                 0.0
pH                      0.0
sulphates               0.0
alcohol                 0.0
quality                 0.0
dtype: float64

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = df.drop(['quality'], axis=1)
y = df['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [7]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier(random_state=42, max_depth=10)
dt_model.fit(X_train, y_train)

c:\Users\killr\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


DecisionTreeClassifier(max_depth=10, random_state=42)

In [8]:
from sklearn.metrics import f1_score
print('f1-score for log reg on test data:', f1_score(lr_model.predict(X_test),y_test))
print('f1-score for dec tree on test data:', f1_score(dt_model.predict(X_test),y_test))

f1-score for log reg on test data: 0.75
f1-score for dec tree on test data: 0.7927927927927927


# БЭГГИНГ

In [9]:
from sklearn.ensemble import BaggingClassifier

bc_model = BaggingClassifier(
    random_state=42,
    estimator=DecisionTreeClassifier(random_state=42, max_depth=10),
    n_estimators=1500
)

bc_model.fit(X_train,y_train)
print('f1-score for bagging model on test data:', f1_score(bc_model.predict(X_test),y_test))

f1-score for bagging model on test data: 0.8181818181818181


### Случайный лес

Одно из важных понятий, которое здесь появляется, — это метод случайных подпространств, который используется для построения ансамблей моделей.

Кратко опишем его принцип:

* Отбираем обучающую выборку.
* Определяем число моделей, которые войдут в ансамбль.
* Для каждой модели берём не все признаки, а только часть из них и формируем выборку с использованием случайно выбранного набора признаков.
* Объединяем все результаты и определяем итоговое решение по объектам.

Обратите внимание на важную особенность: **здесь выбирается не только обучающая выборка, но ещё и случайная выборка из признаков.**

***Алгоритм случайного леса в таком контексте реализуется следующим образом:***

1. Для того чтобы построить i-е дерево леса, из обучающей выборки $X$ берём случайную подвыборку $X_i$ того же размера, что и вся обучающая выборка.
2. После этого в каждой вершине каждого дерева из $M$ возможных признаков выбираем случайную группу признаков объёма $L$. Для выбранных признаков ищем оптимальное разбиение. Рекомендуется использовать $L=\sqrt{M}$ в задачах классификации и $\frac{M}{3}$ — в задачах регрессии.
3. Для получения предсказания необходимо воспользоваться обычным принципом бэггинга: взять усреднённый ответ в случае регрессии или самый популярный класс — для классификации.

***Плюсы***

* Очень высокая точность предсказания (это один из самых эффективных алгоритмов).
* Данные не обязательно масштабировать: случайный лес не чувствителен к масштабированию и в принципе к любым монотонным преобразованиям, поэтому в таких манипуляциях нет смысла.
* Нет чувствительности к выбросам из-за формирования случайных выборок.
* Не требуется тщательно настраивать множество параметров. Даже если реализовывать алгоритм с настройками по умолчанию, результат будет лучше, чем у большинства других моделей.
* Нет склонности к переобучению.
* Позволяет оценить значимость отдельных признаков.
* Может эффективно работать с несбалансированными классами.
* Показывает высокую точность на данных с большим количеством пропусков.

***Минусы***

* Плохо работает с разрежёнными признаками (т. е. с такими, у которых преимущественно нулевые значения). Это может сказываться, к примеру, на качестве анализа текстов.
* Сложно интерпретировать результаты.

In [10]:
b_df = pd.read_csv('boston.csv')
b_df.isnull().sum()

crim_rate         0
zn                0
business          0
river             0
nit_oxiden        0
rooms             0
age               0
dist              0
highways_index    0
tax               0
pup_per_teaс      0
lower             0
target            0
dtype: int64

In [11]:
b_df.head()

,crim_rate,zn,business,river,nit_oxiden,rooms,age,dist,highways_index,tax,pup_per_teaс,lower,target
0,"0,00632",18,"2,31",0,"0,538","6,575","65,2","4,09",1,296,"15,3","4,98",24
1,"0,02731",0,"7,07",0,"0,469","6,421","78,9","4,9671",2,242,"17,8","9,14","21,6"
2,"0,02729",0,"7,07",0,"0,469","7,185","61,1","4,9671",2,242,"17,8","4,03","34,7"
3,"0,03237",0,"2,18",0,"0,458","6,998","45,8","6,0622",3,222,"18,7","2,94","33,4"
4,"0,06905",0,"2,18",0,"0,458","7,147","54,2","6,0622",3,222,"18,7","5,33","36,2"


In [12]:
b_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   crim_rate       506 non-null    object
 1   zn              506 non-null    object
 2   business        506 non-null    object
 3   river           506 non-null    int64 
 4   nit_oxiden      506 non-null    object
 5   rooms           506 non-null    object
 6   age             506 non-null    object
 7   dist            506 non-null    object
 8   highways_index  506 non-null    int64 
 9   tax             506 non-null    int64 
 10  pup_per_teaс    506 non-null    object
 11  lower           506 non-null    object
 12  target          506 non-null    object
dtypes: int64(3), object(10)
memory usage: 51.5+ KB


In [13]:
obj_cols = list(b_df.dtypes[b_df.dtypes == 'object'].index)

for feature in obj_cols:
    b_df[feature] = b_df[feature].apply(lambda x: x.replace(',', '.'))
    b_df[feature] = pd.to_numeric(b_df[feature])
    

In [14]:
X = b_df.drop(['target'], axis=1)
y = b_df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=13)

In [15]:
y_train.values.mean()

22.77316384180791

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [17]:
lr_model = LinearRegression()
lr_model.fit(X_train,y_train)
y_test_pred = lr_model.predict(X_test)
mean_absolute_error(y_test_pred, y_test)

3.722279395856146

In [18]:
dt_model = DecisionTreeRegressor(random_state=13)
dt_model.fit(X_train,y_train)
y_test_pred = dt_model.predict(X_test)
print('ошибка на тестовой выборке:', mean_absolute_error(y_test_pred, y_test) )
print('ошибка на обучающей выборке:', mean_absolute_error(dt_model.predict(X_train), y_train) )

ошибка на тестовой выборке: 2.8388157894736845
ошибка на обучающей выборке: 0.0


In [19]:
for n_trees in [3, 10, 100, 500]:
    rf_model = RandomForestRegressor(random_state=13, n_estimators=n_trees)
    rf_model.fit(X_train,y_train)
    y_test_pred = rf_model.predict(X_test)
    print(f'ошибка на тестовой выборке при {n_trees} деревьях в лесу:', mean_absolute_error(y_test_pred, y_test) )
    

ошибка на тестовой выборке при 3 деревьях в лесу: 2.9328947368421057
ошибка на тестовой выборке при 10 деревьях в лесу: 2.466315789473684
ошибка на тестовой выборке при 100 деревьях в лесу: 2.2559736842105265
ошибка на тестовой выборке при 500 деревьях в лесу: 2.2365657894736892


# БУСТИНГ

ADABOOST

1. Инициируем веса объектов выборки: $w_j = \frac{1}{N}$, для всех наблюдейний (строк) $j = 1,2,...,N$. Т.е. на старте у всех наблюдений равные веса.
2. Пусть $K$ - это число базовых моделей. Тогда для всех $i = 1,2,...,K$ последовательно:
    * строим классификатор $a_i(x)$, используя текущие веса $w_j$.
    * Вычисляем ошибку текущей модели $\operatorname{err}_{i}=\sum_{j=1}^{N} w_{j}\left[y_{j} \neq a_{i}\left(x_{j}\right)\right]$
    * Вычисляем вес текущей модели: $c_{i}=\frac{1}{2} \ln \frac{1-e r r_{i}}{e r r_{i}}$
    * Получаем новые веса объектов (классы определяются как -1 и +1): $w_{j} \leftarrow w_{j} \cdot \exp \left(c_{i}\left[y_{j} \neq a_{i}\left(x_{j}\right)\right]\right), j=1, \ldots, N$
    * Нормируем веса объектов $w_{j} \longleftarrow \frac{w_{j}}{\sum_{j=1}^{N} w_{j}}$
3. Группируем полученные модели $f_{K}(x)=\operatorname{sign}\left[\sum_{i=1}^{K} c_{i} a_{i}(x)\right]$

ГРАДИЕНТНЫЙ БУСТИНГ